In [1]:
import pandas as pd
import os 


In [4]:

init_df = pd.read_csv("C:\\Users\\annaz\\PycharmProjects\\newsalyze-backend_2\\data\\ECBplus-prep\\test_parsing\\2020-05-17_14-59_all.csv", index_col=[0])


In [5]:
DATASET = "dataset"
DOC_ID = "doc_id"
TEXT = "tokens_str"
HEAD = "head"
init_df[DATASET] = ["ecbplus" if "ecbplus" in v else "ecb" for v in list(init_df[DOC_ID].values) ]
init_df[HEAD] = [v.split(" ")[-1] for v in list(init_df[TEXT].values)]

In [7]:

ecbplus_df = init_df[init_df[DATASET] == "ecbplus"]
types = ["HUMAN_PART_PER", "HUMAN_PART_GENERIC", "HUMAN_PART_GPE", "LOC_GEO", "HUMAN_PART_ORG", "ACTION_OCCURRENCE", "NON_HUMAN_PART", 
         "HUMAN_PART_VEH", "LOC_FAC"]
sel_ecbplus_types_df = ecbplus_df[ecbplus_df["full_type"].isin(types)]

In [11]:

# len(sel_ecbplus_types_df)
sel_ecbplus_types_df.to_csv("C:\\Users\\annaz\\PycharmProjects\\newsalyze-backend_2\\data\\ECBplus-prep\\test_parsing\\sel_types.csv")

In [12]:

updated_heads_df = pd.read_csv("C:\\Users\\annaz\\PycharmProjects\\newsalyze-backend_2\\data\\ECBplus-prep\\test_parsing\\sel_types_new.csv", index_col=[0])


In [17]:

import math

WCL, NUMBER, NUMBER_ENT, CONCEPT_TYPE_final = "wcl", "mentions_num", "entities_num","concept_type"
LABEL = "coref_chain"
CONCEPT_TYPE = "full_type"
results_df = pd.DataFrame(columns=[DATASET, CONCEPT_TYPE_final, WCL, NUMBER_ENT, NUMBER])
local_res_df = pd.DataFrame()
for label in set(updated_heads_df[LABEL].values):
    label_df = updated_heads_df[updated_heads_df[LABEL] == label]
    concept = list(set(label_df[CONCEPT_TYPE].values))[0]
    wcl_score = 0
    for head in set(label_df[HEAD].values):
        head_df = label_df[label_df[HEAD] == head]
        wcl_score += len(set(head_df[TEXT].values))/math.log(len(head_df) + 1)
    local_res_df = local_res_df.append(pd.DataFrame({
        CONCEPT_TYPE_final: concept,
        WCL: wcl_score,
        NUMBER: len(label_df)
    }, index=[label]))

sum_df = local_res_df.groupby(by=[CONCEPT_TYPE_final])[NUMBER].sum().reset_index()
avg_df = local_res_df.groupby(by=[CONCEPT_TYPE_final])[WCL].mean().reset_index()
count_df = local_res_df.groupby(by=[CONCEPT_TYPE_final])[WCL].count().reset_index()
results_df[DATASET] = ["ECB"] * len(sum_df)
results_df[CONCEPT_TYPE_final] = sum_df[CONCEPT_TYPE_final].values
results_df[WCL] = avg_df[WCL].values
results_df[NUMBER] = sum_df[NUMBER].values
results_df[NUMBER_ENT] = count_df[WCL].values


In [16]:

results_df.to_csv("C:\\Users\\annaz\\PycharmProjects\\newsalyze-backend_2\\data\\ECBplus-prep\\test_parsing\\wcl_ecb.csv")